# install gymnasium

In [15]:
!pip install gymnasium
!pip install "gymnasium[accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


#import libraries

In [16]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

#Make an brain architerture for AI

In [17]:
class Neural(nn.Module):
  def __init__(self, state_size, action_size, seed = 42 ):
    super(Neural, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

#Setting up the environment

In [18]:
import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
num_action = env.action_space.n
print("State Shape ", state_shape)
print("State Size: ", state_size)
print("number of actions: ", num_action)


State Shape  (8,)
State Size:  8
number of actions:  4


#Hyperparameters

In [19]:
learning_rate = 5e-4
batch_size = 100
discount = 0.99
replay_buffer = int(1e5)
interpolation_parameters = 1e-3

#Implementing Experiences Replay

In [20]:
class Replay(object):

  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batc_size):
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

#Implementing DQN Agent

In [21]:
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_network = Neural(state_size, action_size).to(self.device)
    self.target_network = Neural(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_network.parameters(), lr = learning_rate)
    self.memory = Replay(replay_buffer)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    # The push method likely expects a single experience tuple, not individual arguments
    self.memory.push((state, action, reward, next_state, done))  # Pass arguments as a tuple
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > batch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount)



  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_network.eval()
    with torch.no_grad():
      action_values = self.local_network(state)
    self.local_network.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_network(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (discount * next_q_targets * (1-dones))
    q_expected = self.local_network(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_network, self.target_network, interpolation_parameters)

  def soft_update(self, local_model, target_model, interpolation_parameters):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameters * local_param.data + (1 - interpolation_parameters) * target_param.data)



#Initializing DQN agent

In [22]:
agent = Agent(state_size, num_action)

#Train the ai

In [26]:
num_eps = 2000
max_eps = 1000
start_val = 1.0
end_val = 0.01
decay_val = 0.995
val = start_val
scores_on_100eps = deque(maxlen = 100)

for eps in range(1, num_eps + 1):
  state, _ = env.reset()
  score = 0
  for t in range(max_eps):
    action = agent.act(state, val)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100eps.append(score)
  val = max(end_val, decay_val * val)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(eps, np.mean(scores_on_100eps)), end="")
  if eps % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(eps, np.mean(scores_on_100eps)))
  if np.mean(scores_on_100eps) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(eps - 100, np.mean(scores_on_100eps)))
    break

Episode 100	Average Score: -119.54
Episode 200	Average Score: -24.18
Episode 300	Average Score: 55.11
Episode 379	Average Score: 200.44
Environment solved in 279 episodes!	Average Score: 200.44


#Visualization the result

In [27]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()